In [68]:
:dep aws-config = "0.47.0"
:dep aws-sdk-sagemaker = "0.17.0"
:dep aws-sdk-sagemakerruntime = "0.17.0"
:dep tokio = { version = "1", features = ["full"]}
:dep aws-smithy-types-convert = { version = "0.47.0", features = ["convert-chrono"] }

In [5]:
let config = aws_config::load_from_env().await;

In [43]:
//let client = aws_sdk_sagemaker::Client::new(&config);

# List Training jobs

In [7]:
use aws_smithy_types_convert::date_time::DateTimeExt; // For .to_chrono_utc()

let config = aws_config::load_from_env().await;
let client = aws_sdk_sagemaker::Client::new(&config);
let job_details = client.list_training_jobs().send().await?;

println!("Jobs:");

for j in job_details.training_job_summaries().unwrap_or_default() {
    let name = j.training_job_name().unwrap_or_default();
    let creation_time = j.creation_time().unwrap().to_chrono_utc();
    let training_end_time = j.training_end_time().unwrap().to_chrono_utc();

    let status = j.training_job_status().unwrap();
    let duration = training_end_time - creation_time;

    println!("  Name:               {}", name);
    println!(
        "  Creation date/time: {}",
        creation_time.format("%Y-%m-%d@%H:%M:%S")
    );
    println!("  Duration (seconds): {}", duration.num_seconds());
    println!("  Status:             {}", status.as_ref());
    println!();
}


failed to construct request: Failed to load credentials from the credentials provider: Credentials provider timed out after 5 seconds


In [44]:
use aws_sdk_sagemaker::model::{AlgorithmSpecification as AlgoSpec, TrainingInputMode, TrainingInstanceType, StoppingCondition, OutputDataConfig, ResourceConfig};

let config = aws_config::load_from_env().await;
let client = aws_sdk_sagemaker::Client::new(&config);
let training_job = client.create_training_job()
    .training_job_name("Training-job-from-notebook1")
    .algorithm_specification(
        AlgoSpec::builder()
            .training_image("936391532182.dkr.ecr.eu-central-1.amazonaws.com/rust-sagemaker-train:latest")
            .training_input_mode(TrainingInputMode::File)
            .build()
    )
    .role_arn("arn:aws:iam::936391532182:role/service-role/AmazonSageMaker-ExecutionRole-20211201T154965")
    .stopping_condition(
        StoppingCondition::builder()
            .max_runtime_in_seconds(300)
            .build()
    )
    .output_data_config(
        OutputDataConfig::builder()
            .s3_output_path("s3://random-demo-936391532182/sagemaker-output")
            .build()
    )
    .resource_config(
        ResourceConfig::builder()
            .instance_type(TrainingInstanceType::MlM4Xlarge)
            .instance_count(1)
            .volume_size_in_gb(1)
            .build()
    )
    .send().await?;

println!("{:?}", training_job);

ResourceInUse: Training job names must be unique within an AWS account and region, and a training job with this name already exists (arn:aws:sagemaker:eu-central-1:936391532182:training-job/training-job-from-notebook1)


In [67]:
:clear

In [16]:
println!("{:?}", training_job);

CreateTrainingJobOutput { training_job_arn: Some("arn:aws:sagemaker:eu-central-1:936391532182:training-job/training-job-from-notebook1") }


# TODO: deploy endpoint

# Invoke endpoint

In [69]:
use std::str;
use aws_sdk_sagemakerruntime::types::Blob;

let config = aws_config::load_from_env().await;
let client = aws_sdk_sagemakerruntime::Client::new(&config);
{
let inference_result = client.invoke_endpoint()
    .endpoint_name("rust-model-4") // TODO: laod this from deployment
    .body(Blob::new("1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0".as_bytes()))
    .content_type("text/csv")
    
    .send().await?;
println!("{:?}", String::from_utf8(inference_result.body().unwrap().clone().into_inner()));
}

Ok("{\"prediction\":21.615257}")


()

In [65]:
:linker lld

linker: lld


In [58]:
:sccache

sccache: true
